In [7]:
import os
if os.getcwd()[-len('irl-chess'):] != 'irl-chess':
    os.chdir('../')
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torchmetrics import MeanSquaredError
from project import alpha_beta_search, evaluate_board, get_midgame_boards, load_chess_df
from tqdm import tqdm
import chess
from copy import copy

In [8]:
class SimpleRegressionModel(pl.LightningModule):
    def __init__(self, input_size, output_size, initial_weights=None, lr=1e-3):
        super(SimpleRegressionModel, self).__init__()
        self.lr = lr
        # Define the model architecture
        self.model = nn.Sequential(
            nn.Linear(input_size, output_size)
        )

        # Set initial weights if provided
        if initial_weights is not None:
            self.model[0].weight.data = initial_weights.clone()
        self.loss_function = nn.MSELoss()
        self.mse = MeanSquaredError()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        predictions = self(x)
        loss = self.loss_function(predictions.squeeze(), y.squeeze())
        self.log('train_loss', loss.item(), on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.squeeze()
        predictions = self(x).squeeze()
        loss = self.loss_function(predictions, y)
        self.log('val_loss', loss.item(), on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [9]:
def get_trajectories(boards, R, heuristic=None):
    """
    Return a list of trajectories where each trajectory
    has its heuristic value calculated
    :param boards: 
    :param R: 
    :param heuristic: 
    :return: 
    """
    if heuristic is None:
        heuristic = lambda board: torch.ones_like(R)
    trajectories = torch.empty((len(boards), len(R)), dtype=R.dtype, device=R.device)
    for i, board in enumerate(boards):
        Q, board_, moves = alpha_beta_search(board=board, depth=depth, R=R.cpu().detach().numpy(), )
        trajectories[i] = heuristic(board_)
    return trajectories

In [19]:
n_trajectories = 10
depth = 3
n_epochs = 2
min_elo = 1100
max_elo = 1300
R = torch.rand(6) * 100
model = SimpleRegressionModel(input_size=len(R), output_size=len(R), initial_weights=R.unsqueeze(0).expand(len(R), -1).T)

In [20]:
df = load_chess_df(n_files=3, overwrite=False)
boards_temp = get_midgame_boards(df=df, n_boards=n_trajectories, min_elo=min_elo, max_elo=max_elo)
boards = []
for el in boards_temp:
    boards += el[:1]



-------------------  1/12  -------------------


C:\Users\toell\OneDrive\Documents\GitHub\irl-chess\data\processed\lichess_db_standard_rated_2013-01.csv already exists and was not changed
Time taken: 0.00 seconds for file
Time taken: 0.00 seconds in total


-------------------  2/12  -------------------


C:\Users\toell\OneDrive\Documents\GitHub\irl-chess\data\processed\lichess_db_standard_rated_2013-02.csv already exists and was not changed
Time taken: 0.00 seconds for file
Time taken: 0.00 seconds in total


-------------------  3/12  -------------------


C:\Users\toell\OneDrive\Documents\GitHub\irl-chess\data\processed\lichess_db_standard_rated_2013-03.csv already exists and was not changed
Time taken: 0.00 seconds for file
Time taken: 0.00 seconds in total


Searching for boards:   0%|          | 1095/402536 [00:00<00:12, 33222.87it/s]


In [21]:
for epoch in tqdm(range(n_epochs), desc='Epoch'):
    trajectories = get_trajectories(boards=boards, R=R, )
    
    

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 3, got 2)